# Регрессионный анализ зависимости биржевых индексов от макроэкономических показателей США

### В этом ноутбуке проведен регрессионный анализ зависимости индексов для Англии (UKX Index), Японии (NKY Index) и Китая (HSI Index) от макроэкономических показателей США.

### 1. Подготовка данных

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

us_macro_data_all = pd.read_excel('datamfs.xlsx', sheet_name='US')

us_macro_data_all['Dates'] = pd.to_datetime(us_macro_data_all['Dates'], errors='coerce')

us_macro_data_all.set_index('Dates', inplace=True)
us_macro_data_all_monthly = us_macro_data_all
us_macro_data_all_monthly.dropna(inplace=True)

# Загружаем данные по индексам для Англии, Японии и Китая
bloomberg_data = pd.read_excel('data.xlsx', sheet_name='bloomberg')
bloomberg_data_selected = bloomberg_data[['Dates', 'UKX Index', 'NKY Index', 'HSI Index']]
bloomberg_data_selected['Dates'] = pd.to_datetime(bloomberg_data_selected['Dates'], errors='coerce')

merged_data_with_indexes = pd.merge(bloomberg_data_selected, us_macro_data_all_monthly, on='Dates', how='inner')
merged_data_with_indexes_clean = merged_data_with_indexes.resample('M').last().pct_change().dropna(inplace=True)

X_clean_all = merged_data_with_indexes_clean[['GT10 Govt', 'GT2 Govt', 'FDTR Index', 'GT5 Govt', 'CPI YOY  Index', 'CPI XYOY Index', 'IP Index', 'GDP CQOQ Index']]
y_clean_all = merged_data_with_indexes_clean[['UKX Index', 'NKY Index', 'HSI Index']]

# Рассчитываем корреляцию для всех данных
correlation_matrix_with_indexes_clean = merged_data_with_indexes_clean.corr()
plt.figure(figsize=(12, 8))
plt.imshow(correlation_matrix_with_indexes_clean, cmap='coolwarm', interpolation='none')
plt.colorbar()
for i in range(len(correlation_matrix_with_indexes_clean.columns)):
    for j in range(len(correlation_matrix_with_indexes_clean.columns)):
        plt.text(j, i, f"{correlation_matrix_with_indexes_clean.iloc[i, j]:.2f}",
                 ha='center', va='center', color='black', fontsize=10)

plt.xticks(np.arange(len(correlation_matrix_with_indexes_clean.columns)), correlation_matrix_with_indexes_clean.columns, rotation=45)
plt.yticks(np.arange(len(correlation_matrix_with_indexes_clean.columns)), correlation_matrix_with_indexes_clean.columns)
plt.title("Корреляционная матрица между индексами и макроэкономическими показателями США с отображением значений")
plt.tight_layout()
plt.show()


C:\Users\ggwpn\AppData\Local\Temp\ipykernel_10880\600747092.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bloomberg_data_selected['Dates'] = pd.to_datetime(bloomberg_data_selected['Dates'], errors='coerce')
C:\Users\ggwpn\AppData\Local\Temp\ipykernel_10880\600747092.py:20: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  merged_data_with_indexes_clean = merged_data_with_indexes.resample('M').last().pct_change().dropna(inplace=True)


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

### 2. Построение регрессионных моделей

In [15]:

# Строим регрессию для UK (UKX Index)
y_uk_clean = merged_data_with_indexes_clean['UKX Index']
X_uk_clean = X_clean_all
X_uk_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
X_uk_clean.dropna(inplace=True)
y_uk_clean = y_uk_clean.loc[X_uk_clean.index]
X_uk_clean = sm.add_constant(X_uk_clean)
uk_model_clean = sm.OLS(y_uk_clean, X_uk_clean).fit()

print(uk_model_clean.summary())

                            OLS Regression Results                            
Dep. Variable:              UKX Index   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     1.878
Date:                Sun, 25 Jan 2026   Prob (F-statistic):             0.0692
Time:                        18:31:50   Log-Likelihood:                -1076.1
No. Observations:                 132   AIC:                             2170.
Df Residuals:                     123   BIC:                             2196.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           6554.0206     77.470     84.

C:\Users\ggwpn\AppData\Local\Temp\ipykernel_33260\2727300621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_uk_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\ggwpn\AppData\Local\Temp\ipykernel_33260\2727300621.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_uk_clean.dropna(inplace=True)


In [16]:
# Строим регрессию для Japan (NKY Index)
y_japan_clean = merged_data_with_indexes_clean['NKY Index']
X_japan_clean = X_clean_all
X_japan_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
X_japan_clean.dropna(inplace=True)
y_japan_clean = y_japan_clean.loc[X_japan_clean.index]
X_japan_clean = sm.add_constant(X_japan_clean)
japan_model_clean = sm.OLS(y_japan_clean, X_japan_clean).fit()

print(japan_model_clean.summary())

                            OLS Regression Results                            
Dep. Variable:              NKY Index   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     1.391
Date:                Sun, 25 Jan 2026   Prob (F-statistic):              0.207
Time:                        18:31:53   Log-Likelihood:                -1369.3
No. Observations:                 132   AIC:                             2757.
Df Residuals:                     123   BIC:                             2783.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.908e+04    714.474     26.

C:\Users\ggwpn\AppData\Local\Temp\ipykernel_33260\2054947712.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_japan_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\ggwpn\AppData\Local\Temp\ipykernel_33260\2054947712.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_japan_clean.dropna(inplace=True)


In [17]:
# Строим регрессию для China (HSI Index)
y_china_clean = merged_data_with_indexes_clean['HSI Index']
X_china_clean = X_clean_all
X_china_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
X_china_clean.dropna(inplace=True)
y_china_clean = y_china_clean.loc[X_china_clean.index]
X_china_clean = sm.add_constant(X_china_clean)
china_model_clean = sm.OLS(y_china_clean, X_china_clean).fit()

print(china_model_clean.summary())

                            OLS Regression Results                            
Dep. Variable:              HSI Index   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.218
Date:                Sun, 25 Jan 2026   Prob (F-statistic):              0.294
Time:                        18:31:54   Log-Likelihood:                -1268.1
No. Observations:                 132   AIC:                             2554.
Df Residuals:                     123   BIC:                             2580.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           2.243e+04    331.892     67.

C:\Users\ggwpn\AppData\Local\Temp\ipykernel_33260\2954150666.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_china_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\ggwpn\AppData\Local\Temp\ipykernel_33260\2954150666.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_china_clean.dropna(inplace=True)
